# Import, options

In [73]:
import numpy, matplotlib, pandas, scanpy

In [74]:
matplotlib.rcParams.update({'font.size':20, 'xtick.labelsize':20, 'ytick.labelsize':20, 
                            'axes.grid' : True, 'grid.alpha': 0.5, 'grid.linestyle' : ':',
                            'figure.figsize':(8, 5), 'svg.fonttype' : 'none'})

In [75]:
file_directory = "/Users/kja11/OneDrive - Menntaský/PostDoc_Hypothermia/in_silico/Python/"

# Data Downloads

In [76]:
#data mouse_NPC
mouse_NPC_df = pandas.read_csv(file_directory+'1) input/mNPC_all_detected_genes_DEseq.csv',
                               sep = ',')
print(mouse_NPC_df.shape, '\n', mouse_NPC_df.columns)

mouse_NPC_df = mouse_NPC_df[['gene', 'log2FoldChange', 'pvalue', 'padj', 'lfcSE']]
mouse_NPC_df

(16062, 15) 
 Index(['Row.names', 'baseMean', 'log2FoldChange', 'lfcSE', 'stat', 'pvalue',
       'padj', 'gene', '37_degrees', '37_degrees.1', '37_degrees.2',
       '32_degrees', '32_degrees.1', '32_degrees.2', '32_degrees.3'],
      dtype='object')


,gene,log2FoldChange,pvalue,padj,lfcSE
0,Cirbp,2.021396,4.305259e-145,6.939647e-141,0.078809
1,Xbp1,-1.884029,4.131947e-140,3.330143e-136,0.074767
2,Hspa8,-1.231949,4.968957e-92,2.669821e-88,0.060548
3,Dnaja3,-1.053931,9.850430e-75,3.969477e-71,0.057622
4,Tra2a,-2.158750,2.895976e-68,9.336048e-65,0.123641
...,...,...,...,...,...
16057,Pex2,0.000098,9.991703e-01,9.994484e-01,0.094263
16058,Ank3,-0.000171,9.993065e-01,9.994925e-01,0.196786
16059,Hmgcl,0.000057,9.994949e-01,9.995569e-01,0.090005
16060,Arl6ip5,0.000070,9.994792e-01,9.995569e-01,0.107616


In [30]:
# DL List of the SP1 candidat repressors
SP1_repressors = pandas.read_csv(file_directory+'1) input/SP1_repressors.csv', sep = ',')
SP1_repressors

,Gene Symbol,Score,FDR,GeCKO_number
0,STXBP1,5.7,0.03,1
1,LRRC26,5.2,0.08,3
2,MUC15,5.0,0.08,4
3,GATAD2A,5.0,0.08,5
4,MUT,4.7,0.08,7
...,...,...,...,...
490,CNTNAP2,2.6,0.25,523
491,hsa-mir-5693,0.3,0.25,525
492,hsa-mir-300,2.6,0.25,527
493,ZNF816-ZNF321P,2.6,0.23,531


In [31]:
# DL list human and mouse orthologs
list_hum_mous = pandas.read_csv(file_directory+'human_mouse_1to1_orthologs.csv', sep = ',')
list_hum_mous

,human,mouse
0,A1BG,A1bg
1,A1CF,A1cf
2,A2LD1,A2ld1
3,A2M,A2m
4,A4GALT,A4galt
...,...,...
15542,ZYG11A,Zyg11a
15543,ZYG11B,Zyg11b
15544,ZYX,Zyx
15545,ZZEF1,Zzef1


# Transform to have mice gene symbols 

In [32]:
#rename to be able to merge the two df
SP1_repressors = SP1_repressors.rename(columns = {'Gene Symbol' : 'human'})

# merge the SP1 df with the hum_mouse to have mouse gene symbols
SP1_repressors = SP1_repressors.merge(list_hum_mous)
print(SP1_repressors)

SP1_repressors = SP1_repressors[['mouse', 'Score', 'FDR']]
SP1_repressors = SP1_repressors.rename(columns = {'mouse' : 'gene'})
SP1_repressors

       human  Score   FDR  GeCKO_number     mouse
0     STXBP1    5.7  0.03             1    Stxbp1
1     LRRC26    5.2  0.08             3    Lrrc26
2      MUC15    5.0  0.08             4     Muc15
3    GATAD2A    5.0  0.08             5   Gatad2a
4        MUT    4.7  0.08             7       Mut
..       ...    ...   ...           ...       ...
324    WNT11    2.6  0.25           507     Wnt11
325   R3HDM1    2.6  0.25           508    R3hdm1
326     RGMB    2.6  0.25           510      Rgmb
327  C9orf78    2.6  0.25           522  BC005624
328  CNTNAP2    2.6  0.25           523   Cntnap2

[329 rows x 5 columns]


,gene,Score,FDR
0,Stxbp1,5.7,0.03
1,Lrrc26,5.2,0.08
2,Muc15,5.0,0.08
3,Gatad2a,5.0,0.08
4,Mut,4.7,0.08
...,...,...,...
324,Wnt11,2.6,0.25
325,R3hdm1,2.6,0.25
326,Rgmb,2.6,0.25
327,BC005624,2.6,0.25


In [51]:
# Merge the two tables and only keep the sp1_repressors in the RNAseq
df_mNPC_sp1rep = pandas.merge(mouse_NPC_df, SP1_repressors, on= 'gene')

print(f'number of Sp1 repressors in the RNAseq is {len(df_mNPC_sp1rep)} on a total of {len(SP1_repressors)}')
print(f'{len(SP1_repressors) - len(df_mNPC_sp1rep)} repressors are not in the RNAseq')

df_mNPC_sp1rep

number of Sp1 repressors in the RNAseq is 267 on a total of 329
62 repressors are not in the RNAseq


,gene,log2FoldChange,pvalue,padj,lfcSE,Score,FDR
0,Creld2,-1.055765,4.374648e-44,2.014713e-41,0.075810,2.8,0.23
1,Tfdp2,0.963359,3.843392e-42,1.511015e-39,0.070820,2.7,0.23
2,Noc3l,-1.041630,1.709833e-27,2.375931e-25,0.095879,3.1,0.22
3,Pias3,0.768008,1.526150e-26,1.983872e-24,0.072030,0.0,0.24
4,Mns1,-1.103323,2.663300e-22,2.308050e-20,0.113597,3.9,0.16
...,...,...,...,...,...,...,...
262,Hrk,-0.010509,9.783793e-01,9.890559e-01,0.387757,2.6,0.25
263,Isca2,-0.002630,9.837204e-01,9.915327e-01,0.128887,0.3,0.23
264,Arfgap1,0.000418,9.951379e-01,9.976676e-01,0.068638,2.8,0.23
265,Gorab,0.000556,9.972183e-01,9.987600e-01,0.159352,0.3,0.25


In [52]:
#control values did not change.
print(mouse_NPC_df[mouse_NPC_df['gene'] == 'Tfdp2'])
print(df_mNPC_sp1rep[df_mNPC_sp1rep['gene'] == 'Tfdp2'])

     gene  log2FoldChange        pvalue          padj    lfcSE
40  Tfdp2        0.963359  3.843392e-42  1.511015e-39  0.07082
    gene  log2FoldChange        pvalue          padj    lfcSE  Score   FDR
1  Tfdp2        0.963359  3.843392e-42  1.511015e-39  0.07082    2.7  0.23


# TEST

In [59]:
df_mNPC_sp1rep

,gene,log2FoldChange,pvalue,padj,lfcSE,Score,FDR
0,Creld2,-1.055765,4.374648e-44,2.014713e-41,0.075810,2.8,0.23
1,Tfdp2,0.963359,3.843392e-42,1.511015e-39,0.070820,2.7,0.23
2,Noc3l,-1.041630,1.709833e-27,2.375931e-25,0.095879,3.1,0.22
3,Pias3,0.768008,1.526150e-26,1.983872e-24,0.072030,0.0,0.24
4,Mns1,-1.103323,2.663300e-22,2.308050e-20,0.113597,3.9,0.16
...,...,...,...,...,...,...,...
262,Hrk,-0.010509,9.783793e-01,9.890559e-01,0.387757,2.6,0.25
263,Isca2,-0.002630,9.837204e-01,9.915327e-01,0.128887,0.3,0.23
264,Arfgap1,0.000418,9.951379e-01,9.976676e-01,0.068638,2.8,0.23
265,Gorab,0.000556,9.972183e-01,9.987600e-01,0.159352,0.3,0.25


In [60]:
df_mNPC_sp1rep = df_mNPC_sp1rep[['gene', 'log2FoldChange', 'pvalue', 'padj']]
df_mNPC_sp1rep = df_mNPC_sp1rep.set_index('gene')

df_mNPC_sp1rep.to_csv(file_directory+'1) input/from_output/df_mNPC_sp1rep.tsv',sep = "\t")
df_mNPC_sp1rep

,gene,log2FoldChange,pvalue,padj
0,Creld2,-1.055765,4.374648e-44,2.014713e-41
1,Tfdp2,0.963359,3.843392e-42,1.511015e-39
2,Noc3l,-1.041630,1.709833e-27,2.375931e-25
3,Pias3,0.768008,1.526150e-26,1.983872e-24
4,Mns1,-1.103323,2.663300e-22,2.308050e-20
...,...,...,...,...
262,Hrk,-0.010509,9.783793e-01,9.890559e-01
263,Isca2,-0.002630,9.837204e-01,9.915327e-01
264,Arfgap1,0.000418,9.951379e-01,9.976676e-01
265,Gorab,0.000556,9.972183e-01,9.987600e-01


In [71]:
adata  = scanpy.read_csv(file_directory+'1) input/from_output/df_mNPC_sp1rep.tsv', delimiter='\t')

print(adata.shape)

(267, 3)


In [57]:
nei = 5
scanpy.pp.neighbors(df_mNPC_sp1rep, n_neighbors=nei, n_pcs=100)

AttributeError: 'DataFrame' object has no attribute 'is_view'

In [19]:
%%time

nei = 5
dist = 0.2
spre = 10
scanpy.pp.neighbors(df_final, n_neighbors=nei, n_pcs=100)
scanpy.tl.leiden(adata)
scanpy.tl.paga(adata)
scanpy.pl.paga(adata, plot=False)
scanpy.tl.umap(adata, min_dist=dist, spread=spre, init_pos='paga')      
scanpy.pl.umap(adata, color='leiden', palette=two_hundred_colors, legend_loc=None, save='_all')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\kja11\anaconda3\Lib\site-packages\IPython\core\magics\execution.py", line 1340, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 4, in <module>
  File "C:\Users\kja11\anaconda3\Lib\site-packages\scanpy\neighbors\__init__.py", line 145, in neighbors
    if adata.is_view:  # we shouldn't need this here...
       ^^^^^^^^^^^^^
  File "C:\Users\kja11\anaconda3\Lib\site-packages\pandas\core\generic.py", line 5989, in __getattr__
    return object.__getattribute__(self, name)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'DataFrame' object has no attribute 'is_view'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\kja11\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\kja11\an